# Paragraph analysis

## Imports

In [1]:
import re
import pandas as pd

## Load text file

In [2]:
df = pd.read_csv("text.txt", header=None, sep='\n')
df.to_string().lower()

"                                                                                                           0\n0      as a term, data analytics predominantly refers to an assortment of applications, from basic business \n1         intelligence (bi), reporting and online analytical processing (olap) to various forms of advanced \n2         analytics. in that sense, it's similar in nature to business analytics, another umbrella term for \n3   approaches to analyzing data -- with the difference that the latter is oriented to business uses, while \n4       data analytics has a broader focus. the expansive view of the term isn't universal, though: in some \n5       cases, people use data analytics specifically to mean advanced analytics, treating bi as a separate \n6          category.  data analytics initiatives can help businesses increase revenues, improve operational \n7            efficiency, optimize marketing campaigns and customer service efforts, respond more quickly to \n8        

## 3a. What is the probability of the word “data” occurring in each line?

## 3b. What is the distribution of distinct word counts across all the lines?

## 3c. What is the probability of the word “analytics” occurring after the word “data”?